This notebook illustrates the workflow to calculate variogram of 

In [68]:
# Add python script files to the system env
import sys  
sys.path.insert(1, "..\\My Functions")
# 
import numpy as np
import pandas as pd
import rasterio as rio
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
# Import the custom python script files
import my_Variogram
import my_RomanMetrics
import my_Variogram_Plot

In [69]:
# \\DISC
cwd = "..\\..\\"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [70]:
cwd_Image_Folder = cwd + "\\Sentinel-2 Images Processed\\US-LIN"

In [71]:
# Set g
g = 4.66

In [72]:
# Set internal distance
distances_Int = [30,30,100,300,300,300]
# Set external distance
distances_Ext = [100,300,300,600,900,1200]

In [73]:
# Get the unique distances so we can calculate the variogram first
list_distances = list(set(distances_Int + distances_Ext))
list_distances.sort()
list_distances

[30, 100, 300, 600, 900, 1200]

In [74]:
list_std = []
list_cv = []
list_range = []
list_nugget = []
list_sill = []
list_bins = []
list_experimental_model = []
list_fitted_model = []
list_empiral_X = []
list_empiral_Y = []
for i in range(len(list_distances)):
    img = rio.open(cwd_Image_Folder + "\\NIRv " + str(list_distances[i]) + "m.tif")
    band = img.read(1).reshape(-1)
    temp_std = my_Variogram.calc_STD(band)
    temp_cv = my_Variogram.calc_CV(band)
    coordinates = my_Variogram.get_Coordinates_from_Rio(img)
    temp_range, temp_nugget, temp_sill, temp_bins, temp_experimental_model, temp_fitted_model, temp_empiral_X, temp_empiral_Y = my_Variogram.calc_Variogram_Parameters(band,coordinates,list_distances[i])
    list_std.append(temp_std)
    list_cv.append(temp_cv)
    list_range.append(temp_range)
    list_nugget.append(temp_nugget)
    list_sill.append(temp_sill)
    list_bins.append(temp_bins)
    list_experimental_model.append(temp_experimental_model)
    list_fitted_model.append(temp_fitted_model)
    list_empiral_X.append(temp_empiral_X)
    list_empiral_Y.append(temp_empiral_Y)
    print(f"Distance {list_distances[i]} has been calculated!")

c:\ProgramData\anaconda3\envs\DISC\Lib\site-packages\skgstat\Variogram.py:1715: OptimizeWarning: Covariance of the parameters could not be estimated
  self.cof, self.cov = curve_fit(


Distance 30 has been calculated!
Distance 100 has been calculated!
Distance 300 has been calculated!
Distance 600 has been calculated!
Distance 900 has been calculated!
Distance 1200 has been calculated!


In [75]:
# Make a dataframe to illustrate the numeric result of variograms
df_Vario = pd.DataFrame({
    "Distance": list_distances,
    "STD": list_std,
    "CV": list_cv,
    "Range": list_range,
    "Sill": list_sill,
    "Nugget": list_nugget
})
df_Vario

,Distance,STD,CV,Range,Sill,Nugget
0,30,0.844591,0.008956,20.000000,0.626820,3.118349e-01
1,100,1.803579,0.019019,65.576055,3.883706,6.266085e-31
2,300,5.390128,0.057523,210.000000,31.142912,6.796741e+00
3,600,4.484857,0.048321,157.755668,21.122576,4.401695e+00
4,900,18.635796,0.211051,630.000000,347.913864,1.051105e+02
5,1200,31.965300,0.358960,836.615153,1041.065977,3.253057e+02


In [76]:
list_Rcv = []
list_RAW_Score = []
list_Rse = []
list_Rst = []
list_Rsv = []
list_ST_Score = []
for i in range(len(list_distances)):
    dist_Int = distances_Int[i]
    dist_Ext = distances_Ext[i]
    index_Int = list_distances.index(dist_Int)
    index_Ext = list_distances.index(dist_Ext)
    range_Int, nugget_Int, sill_Int, bins_Int, experimental_model_Int, fitted_model_Int = list_range[index_Int], list_nugget[index_Int], list_sill[index_Int], list_bins[index_Int], list_experimental_model[index_Int], list_fitted_model[index_Int]
    range_Ext, nugget_Ext, sill_Ext, bins_Ext, experimental_model_Ext, fitted_model_Ext = list_range[index_Ext], list_nugget[index_Ext], list_sill[index_Ext], list_bins[index_Ext], list_experimental_model[index_Ext], list_fitted_model[index_Ext]
    cv_Int = list_cv[index_Int]
    cv_Ext = list_cv[index_Ext]
    semivar_range_Exp_Int = my_Variogram.get_Semivar_Exp(dist_Int, bins_Int, experimental_model_Int)
    semivar_range_Exp_Ext = my_Variogram.get_Semivar_Exp(dist_Ext, bins_Ext, experimental_model_Ext)
    # Calculate Rcv
    Rcv = my_RomanMetrics.calc_Rcv(cv_Ext, cv_Int)
    # Calculate RAW Score
    RAW_Score = my_RomanMetrics.calc_RAWScore(Rcv)
    # Calculate Rse
    Rse = my_RomanMetrics.calc_Rse(g, range_Ext, range_Int)
    # Calculate Rst
    Rst = my_RomanMetrics.calc_Rst(my_RomanMetrics.calc_ST(semivar_range_Exp_Ext, nugget_Ext), my_RomanMetrics.calc_ST(semivar_range_Exp_Int, nugget_Int))
    # Calculate Rsv
    Rsv = my_RomanMetrics.calc_Rsv(my_RomanMetrics.calc_SV(fitted_model_Ext, range_Ext, nugget_Ext, sill_Ext), my_RomanMetrics.calc_SV(fitted_model_Int, range_Int, nugget_Int, sill_Int))
    # Calculate ST Score
    ST_Score = my_RomanMetrics.calc_STScore(Rcv, Rse, Rst, Rsv)
    list_Rcv.append(Rcv)
    list_RAW_Score.append(RAW_Score)
    list_Rse.append(Rse)
    list_Rst.append(Rst)
    list_Rsv.append(Rsv)
    list_ST_Score.append(ST_Score)
    print(f"Internal distance {dist_Int} and external distance {dist_Ext} has been computed!")


Internal distance 30 and external distance 100 has been computed!
Internal distance 30 and external distance 300 has been computed!
Internal distance 100 and external distance 300 has been computed!
Internal distance 300 and external distance 600 has been computed!
Internal distance 300 and external distance 900 has been computed!
Internal distance 300 and external distance 1200 has been computed!


In [77]:
# Make a dataframe to illustrate the numeric result of roman metrics
df_Roman = pd.DataFrame({
    "Internal Distance": distances_Int,
    "External Distance": distances_Ext,
    "Rcv": list_Rcv,
    "Rse": list_Rse,
    "Rst": list_Rst,
    "Rsv": list_Rsv,
    "RAW Score": list_RAW_Score,
    "ST Score": list_ST_Score
})
df_Roman

,Internal Distance,External Distance,Rcv,Rse,Rst,Rsv,RAW Score,ST Score
0,30,100,1.123471,0.783804,0.990000,5.524818,0.445049,0.300309
1,30,300,5.422610,0.791319,0.583290,15.334800,0.092207,0.126504
2,100,300,2.024581,0.928257,-0.204377,1.503488,0.246965,0.460319
3,300,600,-0.159978,0.963728,0.004123,-0.239312,3.125431,0.910581
4,300,900,2.668961,0.976881,-0.088192,1.678137,0.187339,0.407280
5,300,1200,5.240244,0.977381,-0.106323,2.503679,0.095415,0.278231


In [78]:
df_Vario.to_csv(cwd_Image_Folder + "\\Vario.csv", index = False)
df_Roman.to_csv(cwd_Image_Folder + "\\Roman.csv", index = False)

In [81]:
my_Variogram_Plot.plot_Variogram(list_distances, list_bins, list_fitted_model, list_range, list_sill, list_nugget, list_empiral_X,list_empiral_Y, site_Name_Graph = cwd_Image_Folder.split("\\")[-1], path_SaveFig = cwd_Image_Folder + "\\Variogram.png", bool_SaveFig = True, bool_CloseFig = True)